## PERFORMANCE EVALUATION


- General testing procedure 
- Performance measures
- k-fold cross-validation


In [1]:
### General testing procedure:

In [2]:
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

Validation using training data: Wrong! 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X, y)
y_pred = model.predict(X)
from sklearn.metrics import accuracy_score
accuracy_score(y, y_pred)

1.0

Split data in training & test subsets and validate using test: 

In [ ]:
from sklearn.model_selection import train_test_split
# split the data with 50% in each set
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=0,
                                  train_size=0.5)

# fit the model on one set of data
model.fit(Xtrain, ytrain)

# evaluate the model on the second set of data
ytest_pred = model.predict(Xtest)
accuracy_score(ytest, ytest_pred)

0.9066666666666666

### Performance measures:

confusion matrix.png![image.png](attachment:image.png)

Classification report:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(ytest, ytest_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        21
           1       0.85      0.93      0.89        30
           2       0.90      0.79      0.84        24

    accuracy                           0.91        75
   macro avg       0.92      0.91      0.91        75
weighted avg       0.91      0.91      0.91        75



Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(ytest, ytest_pred)

array([[21,  0,  0],
       [ 0, 28,  2],
       [ 0,  5, 19]])

Description of the available performance scores in sklearn libraries:
https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

### k-fold Cross-validation

kfold.gif![image.png](attachment:image.png)

In [ ]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(model, iris.data, iris.target, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.96 (+/- 0.05)


In [ ]:
from sklearn import metrics
f1_scores = cross_val_score(model,X,y, cv=5, scoring='f1_macro')
print("Accuracy: %0.2f (+/- %0.2f)" % (f1_scores.mean(), f1_scores.std() * 2))

Accuracy: 0.96 (+/- 0.05)


k-fold cross-validation:

In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)


TRAIN: [ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149] TEST: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74]
TRAIN: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74] TEST: [ 75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92
  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 1

In [ ]:
kf1 = KFold(n_splits=5)
cross_val_score(model, X, y, cv=kf1,scoring='accuracy')

array([1.   , 1.   , 0.867, 0.933, 0.833])

Custom performance metrics

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
def tn(y, y_pred): return confusion_matrix(y, y_pred)[0, 0]
def fp(y, y_pred): return confusion_matrix(y, y_pred)[0, 1]
def fn(y, y_pred): return confusion_matrix(y, y_pred)[1, 0]
def tp(y, y_pred): return confusion_matrix(y, y_pred)[1, 1]
tn_score = make_scorer(tn)
fp_score = make_scorer(fp)
fn_score = make_scorer(fn)
tp_score = make_scorer(tp)

In [ ]:
[tn(ytest,ytest_pred)
,fp(ytest,ytest_pred)
,fn(ytest,ytest_pred)
,tp(ytest,ytest_pred)]

[21, 0, 0, 28]

In [ ]:
tp_score(model,X,y)

48

Multiple performance measures

In [ ]:
from sklearn.model_selection import cross_validate
custom_scores = {'tp': tp_score, 'tn': tn_score,'fp': fp_score, 'fn': fn_score}
cv_results = cross_validate(model,X,y,cv=10,scoring=custom_scores,return_train_score=True)

cv_results

{'fit_time': array([0.001, 0.   , 0.001, 0.001, 0.001, 0.   , 0.   , 0.   , 0.   ,
        0.   ]),
 'score_time': array([0.01 , 0.006, 0.006, 0.007, 0.011, 0.008, 0.007, 0.006, 0.006,
        0.006]),
 'test_tp': array([5, 5, 5, 5, 3, 5, 4, 5, 5, 5]),
 'train_tp': array([45, 45, 45, 45, 45, 45, 45, 45, 45, 45]),
 'test_tn': array([5, 5, 5, 5, 5, 5, 5, 5, 5, 5]),
 'train_tn': array([45, 45, 45, 45, 45, 45, 45, 45, 45, 45]),
 'test_fp': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'train_fp': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'test_fn': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'train_fn': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}

**Exercise: Understanding different performance measures:**

Take a look to the following article discussing how different performance measures capture different 

types of errors in classification problems: 

https://medium.com/datadriveninvestor/accuracy-trap-pay-attention-to-recall-precision-f-score-auc-d02f28d3299c

Use the data in the example in order to create a MATLAB/Python code that computes the confusion matrix and the Reciever-Operating-Characteristic curve (ROC). Upload the codes so that we can share them with your class mates. 

The main objective of this activity is that you get a critical understanding of the different measures and how do they actually capture the relevant errors in your specific problem.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import math
import pandas as pd
import seaborn as sns
import sklearn as skl
from sklearn.decomposition import PCA

# Data vector
# Read .txt document (file has to be in the same folder as this program)
data = pd.read_csv('Data_article.txt', sep=",", header = None)
position = data[:][0]
actualclass = data[:][1]
score = data[:][2]

# Set a threshold to obtain the confusion matrix
# The same results can be obtained using "sklearn" library
string = input("Set your threshold: ")
thresh = float(string)
TP = 0
FP = 0
FN = 0
TN = 0
for i in range(0, np.size(position)):
    # Actual class: POSITIVE
    # Supposed class: POSITIVE
    if ( (score[i] >= thresh) & (actualclass[i] == 1) ):
        confvect = 'TP'
        TP = TP + 1
    # Actual class: NEGATIVE
    # Supposed class: POSITIVE
    elif ( (score[i] >= thresh) & (actualclass[i] == 0) ):
        confvect = 'FP'
        FP = FP + 1
    # Actual class: POSITIVE
    # Supposed class: NEGATIVE
    elif ( (score[i] < thresh) & (actualclass[i] == 1) ):
        confvect = 'FN'
        FN = FN + 1
    # Actual class: NEGATIVE
    # Supposed class: NEGATIVE
    elif ( (score[i] < thresh) & (actualclass[i] == 0) ):
        confvect = 'TN'
        TN = TN + 1

# Model evaluation indicators
Accuracy = (TP+TN)/(TP+TN+FP+FN)
Recall = TP/(TP+FN)
Precision = TP/(TP+FP)
Fscore = (2*Recall*Precision)/(Recall+Precision)
print([Accuracy, Recall, Precision, Fscore])

Set your threshold: 0.1
[0.15, 1.0, 0.15, 0.2608695652173913]
